# Homework 10

## Problem 1. 30 POINTS
Create a bash shell script that will create a sqlite database of the 4 csv files in this directory. Make sure that you assing meaningful data types and constraints to your database.

In [1]:
#please don't run the cell because I have already created .sh file and .db file using these codes
"""
%%bash
cat >> data4.sh << EOF
> #!/bin/bash
> sqlite3 data4.db << eof
> CREATE TABLE billing(
>   "billingid" TEXT UNIQUE NOT NULL PRIMARY KEY,
>   "encounterid" TEXT,
>   "code" TEXT
> );
> CREATE TABLE encounter(
>   "encounterid" TEXT UNIQUE NOT NULL PRIMARY KEY,
>   "patientid" TEXT,
>   "encounterdate" TEXT,
>   "encounterphysician" TEXT,
>   "encounterreason" TEXT,
>   "billingamount"  
> );
> CREATE TABLE patient(
>   "patientid" TEXT UNIQUE NOT NULL PRIMARY KEY,
>   "lastname" TEXT,
>   "firstname" TEXT,
>   "gender" TEXT,
>   "dob" TEXT
> );
> CREATE TABLE physician(
>   "physicianid" TEXT UNIQUE NOT NULL PRIMARY KEY,
>   "lastname" TEXT,
>   "firstname" TEXT,
>   "specialty" TEXT
> );
> .mode csv
> .import billing.csv billing
> .import encounter.csv encounter
> .import patient.csv patient
> .import physician.csv physician
> .schema billing
> .schema encounter
> .schema patient
> .schema physician
> eof
> EOF
chmod 744 data4.sh
./data4.sh
"""

/Users/sinclair/bios821/home_work/home_work/homework10


In [41]:
%%bash
ls -al

total 256
drwxr-xr-x  12 sinclair  staff    384 Nov 26 17:55 .
drwxr-xr-x  13 sinclair  staff    416 Nov 24 18:03 ..
-rw-r--r--@  1 sinclair  staff   6148 Nov 26 17:55 .DS_Store
drwxr-xr-x   3 sinclair  staff     96 Nov 25 16:43 .ipynb_checkpoints
-rw-r--r--   1 sinclair  staff  18283 Nov 26 17:55 Homework10.ipynb
-rw-r--r--   1 sinclair  staff    241 Nov 24 18:03 billing.csv
-rw-r--r--   1 sinclair  staff  36864 Nov 25 18:19 data4.db
-rwxr--r--   1 sinclair  staff    823 Nov 25 18:18 data4.sh
-rw-r--r--   1 sinclair  staff    368 Nov 24 18:03 encounter.csv
-rw-r--r--@  1 sinclair  staff  44014 Nov 26 17:55 graph.pdf
-rw-r--r--   1 sinclair  staff    152 Nov 24 18:03 patient.csv
-rw-r--r--   1 sinclair  staff    141 Nov 24 18:03 physician.csv


## Problem 2. 30 POINTS.
Using python, answer the following questions:
    1. Provide code that creates a pandas dataframe that contains a patient-level dataset with:
        - LastName
        - FirstName
        - Total Billing Amount
    2. The billing department needs to modify each billing statment so that we take the total billing amount for each person, multiply it by 4.2 and subtract 10. Create a new column on your pandas dataframe that applies this correction and name this variable `correction`.
    3. For all patients less than 40 years old, create a pandas data frame that contains:
        - LastName
        - FirstName
        - physician last name
        - physician first name
        - total times seen that physician
    

In [42]:
import pandas as pd
from sqlalchemy import create_engine
engine = create_engine('sqlite:///data4.db')
df1 = pd.read_sql('select * from patient;', engine)
df2 = pd.read_sql('select * from encounter;', engine)
df1 = df1.drop([0])
df2 = df2.drop([0])
df1 = df1.drop(columns = ['gender', 'dob'])
df2 = df2.drop(columns = ['encounterid', 'encounterdate', 'encounterphysician', 'encounterreason'])
df2 = df2.groupby(['patientid'])['billingamount'].sum()
df2 = df2.to_frame()
df = pd.merge(df1, df2, on = 'patientid', how = 'outer')
df['billingamount'] = df['billingamount'].apply(lambda x: 0.00 if pd.isna(x) else float(x))
df.set_index(['patientid'])


,lastname,firstname,billingamount
patientid,,,
1,doe,john,186.97
2,doe,jane,0.00
3,sickly,joseph,888.10
4,well,sally,0.00
5,daily,bob,375.69


In [43]:
df['correction'] = df['billingamount'].apply(lambda x: 4.2 * x - 10 if x != 0 else x)
df

,patientid,lastname,firstname,billingamount,correction
0,1,doe,john,186.97,775.274
1,2,doe,jane,0.00,0.000
2,3,sickly,joseph,888.10,3720.020
3,4,well,sally,0.00,0.000
4,5,daily,bob,375.69,1567.898


In [44]:
from datetime import datetime
from dateutil import relativedelta
import numpy as np
df1 = pd.read_sql('select * from patient;', engine)
df1 = df1.drop([0])
df1 = df1.rename(columns = {'lastname': 'patientlastname', 'firstname': 'patientfirstname'})
df2 = pd.read_sql('select * from encounter;', engine)
df2 = df2.drop([0])
df2['times'] = 1
df2 = df2.rename(columns = {'encounterphysician': 'physicianid'})
df3 = pd.read_sql('select * from physician;', engine)
df3 = df3.drop([0])
df3 = df3.rename(columns = {'lastname': 'physicianlastname', 'firstname': 'physicianfirstname'})
df = pd.merge(df1, df2, on = 'patientid', how = 'outer')
df = pd.merge(df, df3, on = 'physicianid', how = 'outer')
df = df.drop(columns = ['patientid', 'physicianid', 'gender', 'encounterid', 'encounterdate', 'encounterreason', 'billingamount', 'specialty'])
df['times'] = df['times'].apply(lambda x: 0 if pd.isna(x) else x)
df['physicianlastname'] = df['physicianlastname'].apply(lambda x: 'None' if pd.isna(x) else x)
df['physicianfirstname'] = df['physicianfirstname'].apply(lambda x: 'None' if pd.isna(x) else x)
df['dob'] = pd.to_datetime(df['dob'], format = '%m/%d/%y')
df['dob'] = df['dob'].apply(lambda x: x if x.year <= 2018 else x.replace(year = x.year - 100))
now = datetime.now()
df['age'] = df['dob'].apply(lambda x: int(relativedelta.relativedelta(now, x).years) if True else x)
df = df.loc[df['age'] < 40]
df['times'] = df['times'].astype(int)
df = df.groupby(['patientlastname', 'patientfirstname', 'physicianlastname', 'physicianfirstname'])['times'].sum()
df = df.to_frame()
df


times
patientlastname patientfirstname physicianlastname physicianfirstname       
daily           bob              na                li                      1
                                 satterwhite       jill                    1
sickly          joseph           na                li                      3
                                 wei               liu                     2
                                 whitecoat         bill                    1
well            sally            None              None                    0

## Problem 1. 15 POINTS
You should have received an email that contains:
    - host
    - user
    - password
    
Go ahead and create three environment variables that hold this information. From this point forward, if you need this information, you should refer to the environment variables and NOT the values themselves. You will lose all points, if the values themselves are in the code and submitted to github. You will have effectively disclosed very sensitive data. **NOTE**, there is no code you turn in to get points for this problem. Rather, I will know that you have done this by reviewing the code below.

## Problem 2. 5 POINTS
Create a `sqlalchemy` engine that is connected to our database. **NOTE** use the environment variables set up in problem 1 to do this. There should be NO host, username, or passwords visible if you are using environment variables correctly. You may receive some `ImportError`. You will need to figure out how to resolve these issues.

In [45]:
import os
from sqlalchemy import create_engine
engine2 = create_engine("postgresql://"+os.environ.get("user")+":"+os.environ.get("password")+"@"+os.environ.get("host"))


## Problem 3. 20 POINTS
Using the engine created in Problem 2. Use sqlalchemy to help you create a hand drawn ER diagram. Please find a way to get you hand-drawn ER diagram into this repository. HINT, if you take a picture with your phone, you can use that image (which is just a file) to put into this directory.

In [46]:
from sqlalchemy import MetaData
from sqlalchemy_schemadisplay import create_schema_graph

m = MetaData(engine2)
graph = create_schema_graph(metadata=m,font=20*20,
                 show_datatypes=False,
                 show_indexes=False,
                 rankdir='LR',
                 concentrate=False)
graph.write_pdf('graph.pdf')